# Python MySQL

- CRUD Operation

In [5]:
import json
import pymysql

In [6]:
with open('mysql.json', 'r') as file:
    config_str = file.read()
config = json.loads(config_str)

In [7]:
conn = pymysql.connect(
    host = config['host'],
    user = config['user'],
    password = config['password'],
    database = config['database'],
    port = config['port'] 
)

- 테이블 생성

In [8]:
sql_user = """
    CREATE TABLE if NOT EXISTS users (
        uid VARCHAR(20) NOT NULL PRIMARY KEY,
        pwd CHAR(44),
        uname VARCHAR(20) NOT NULL,
        reg_date DATETIME DEFAULT CURRENT_TIMESTAMP,
        is_deleted INT DEFAULT 0
    );
"""

In [9]:
cur = conn.cursor()
cur.execute(sql_user)

0

- 테이블 변경

In [10]:
cur.execute('alter table users add column email varchar(40);')

0

- 데이터 입력

In [11]:
cur = conn.cursor()
cur.execute("INSERT INTO users(uid, uname) VALUES('admin','관리자');")

1

In [12]:
cur.execute('select * from users;')
rows = cur.fetchall()
for row in rows:
    print(row)

('admin', None, '관리자', datetime.datetime(2021, 6, 9, 17, 17, 22), 0, None)


In [13]:
# 입력한 데이터를 즉시 flush 하려면 conn.commit() 추가
sql_insert_multi = """
    INSERT INTO users(uid, uname) 
        VALUES('eskim','김은숙'),('wjlee','이우정');
"""
cur.execute(sql_insert_multi)
conn.commit()

In [14]:
sql_select = """
    SELECT uid, uname, 
    DATE_FORMAT(reg_date, '%Y-%m-%d %h:%i') AS reg_date,
    is_deleted, email FROM users;
"""
cur.execute(sql_select)
rows = cur.fetchall()
for row in rows:
    print(row)

('admin', '관리자', '2021-06-09 05:17', 0, None)
('eskim', '김은숙', '2021-06-09 05:17', 0, None)
('wjlee', '이우정', '2021-06-09 05:17', 0, None)


In [15]:
# 파라메터 이용, placeholder
uid = 'djy'
uname = '대조영'
cur = conn.cursor()
sql = "INSERT INTO users(uid, uname) VALUES(%s, %s);"
cur.execute(sql, (uid, uname))
conn.commit()

In [16]:
uid = 'gdhong'
cur = conn.cursor()
sql = "INSERT INTO users(uid, uname) VALUES(%s, '홍길동');"
cur.execute(sql, (uid,))
conn.commit()

In [17]:
users = (('jbpark','박재범'),('vincenzo','빈센조'))
cur = conn.cursor()
sql = "INSERT INTO users(uid, uname) VALUES(%s, %s);"
# 아래 for 문은 cur.executemany(sql, users) 로 대체 가능
for user in users:
    cur.execute(sql, user)
conn.commit()

- 데이터 삭제

In [18]:
cur = conn.cursor()
sql = "DELETE FROM users WHERE uid != 'admin';"
cur.execute(sql)
conn.commit()

In [19]:
import pandas as pd
uids = ['djy','eskim','gdhong','jbpark','vincenzo','wjlee']
unames = ['대조영','김은숙','홍길동','박재범','빈센조','이우정']
df = pd.DataFrame({'uid':uids, 'uname':unames})
df

,uid,uname
0,djy,대조영
1,eskim,김은숙
2,gdhong,홍길동
3,jbpark,박재범
4,vincenzo,빈센조
5,wjlee,이우정


In [20]:
cur = conn.cursor()
sql = "INSERT INTO users(uid, uname) VALUES(%s, %s);"
for i in df.index:
    cur.execute(sql, (df.uid[i], df.uname[i]))
conn.commit()

- 데이터 수정

In [21]:
email = 'admin@korea.com'
uid = 'admin'
cur = conn.cursor()
sql = "UPDATE users SET email=%s WHERE uid=%s;"
cur.execute(sql, (email, uid))
conn.commit()

In [22]:
cur = conn.cursor()
sql = "UPDATE users SET email=%s WHERE uid=%s;"
for i in df.index:
    uid = df.uid[i]
    email = f'{uid}@email.com'
    cur.execute(sql, (email, uid))
conn.commit()

- 데이터 조회

In [24]:
sql = """
    SELECT uid, uname, email,
    DATE_FORMAT(reg_date, '%Y-%m-%d %h:%i') AS reg_date
    FROM users WHERE is_deleted=0;
"""

In [25]:
# 한 레코드 조회
cur = conn.cursor()
cur.execute(sql)
result = cur.fetchone()
result

('admin', '관리자', 'admin@korea.com', '2021-06-09 05:17')

In [26]:
# n개의 레코드 조회
cur.fetchmany(2)

(('djy', '대조영', 'djy@email.com', '2021-06-09 05:18'),
 ('eskim', '김은숙', 'eskim@email.com', '2021-06-09 05:18'))

In [27]:
# 모두 조회
cur = conn.cursor()
cur.execute(sql)
results = cur.fetchall()
for row in results:
    print(row)

('admin', '관리자', 'admin@korea.com', '2021-06-09 05:17')
('djy', '대조영', 'djy@email.com', '2021-06-09 05:18')
('eskim', '김은숙', 'eskim@email.com', '2021-06-09 05:18')
('gdhong', '홍길동', 'gdhong@email.com', '2021-06-09 05:18')
('jbpark', '박재범', 'jbpark@email.com', '2021-06-09 05:18')
('vincenzo', '빈센조', 'vincenzo@email.com', '2021-06-09 05:18')
('wjlee', '이우정', 'wjlee@email.com', '2021-06-09 05:18')


In [28]:
# 모두 조회한 뒤 데이터프레임으로 만들기
uids, unames, emails, regs = [],[],[],[]
for row in results:
    uids.append(row[0])
    unames.append(row[1])
    emails.append(row[2])
    regs.append(row[3])
df2 = pd.DataFrame({
    'uid': uids, 'uname': unames,
    'email': emails, 'reg_date': regs
})
df2

,uid,uname,email,reg_date
0,admin,관리자,admin@korea.com,2021-06-09 05:17
1,djy,대조영,djy@email.com,2021-06-09 05:18
2,eskim,김은숙,eskim@email.com,2021-06-09 05:18
3,gdhong,홍길동,gdhong@email.com,2021-06-09 05:18
4,jbpark,박재범,jbpark@email.com,2021-06-09 05:18
5,vincenzo,빈센조,vincenzo@email.com,2021-06-09 05:18
6,wjlee,이우정,wjlee@email.com,2021-06-09 05:18


- 데이터 검색

In [29]:
keyword = '김'
sql = """
    SELECT uid FROM users 
    WHERE is_deleted=0 and uname like %s;
"""

In [30]:
# 결과가 하나라도 튜플로 데이터를 반환
cur = conn.cursor()
cur.execute(sql, keyword+'%')
result = cur.fetchone()
result

('eskim',)

In [31]:
result[0]

'eskim'

In [32]:
cur = conn.cursor()
cur.execute(sql, keyword+'%')
result = cur.fetchall()
result

(('eskim',),)

In [33]:
result[0][0]

'eskim'

In [34]:
cur.close()
conn.close()